In [7]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os
from langgraph.types import Command
from langgraph.prebuilt import create_react_agent
from langchain.tools import tool
from typing_extensions import Literal
from langgraph.graph import MessagesState, StateGraph, START, END
from typing import Annotated, Literal, List, Dict, Any
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults

In [8]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI()
llm.invoke("Hey there, how are you?")

AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to chat with you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 14, 'total_tokens': 48, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BoLQ6PUWkqY4piTjmukEduIG4wiiU', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--14b97710-327f-4a62-8681-64dbb667c300-0', usage_metadata={'input_tokens': 14, 'output_tokens': 34, 'total_tokens': 48, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [9]:
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
search_tool=TavilySearchResults(tavily_api_key=TAVILY_API_KEY)

In [ ]:
members=["researcher","coder"]
options = members+["FINISH"]

In [6]:
class Router(TypedDict):
    next: Literal["researcher", "coder", "FINISH"]

In [ ]:
class State(MessagesState):
    next:str